In [1]:
%pip install pykrx pymysql mysql-connector-python sqlalchemy

In [4]:
import os
from dotenv import load_dotenv
from pykrx import stock
from pykrx import bond

load_dotenv()

KIS_APP_KEY = os.getenv("KIS_APP_KEY")
KIS_APP_SECRET = os.getenv("KIS_APP_SECRET")
KIS_ACC_NO = os.getenv("KIS_ACC_NO")
DB_HOST = os.getenv("DB_HOST")
DB_USERNAME = os.getenv("DB_USERNAME")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_SCHEME = os.getenv("DB_SCHEME")


In [10]:
from pykrx import stock
stock.get_previous_business_days(year=2024, month=10)

[Timestamp('2024-10-02 00:00:00'), Timestamp('2024-10-04 00:00:00')]

In [120]:
import pandas as pd
import requests as rq
from io import StringIO


def get_krx_index_otp(otp_params, headers):
  krx_gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
  krx_otp = rq.post(krx_gen_otp_url, otp_params, headers = headers).text
  return krx_otp

def download_krx_index(at_date, market = None):
  if not market:
    kospi_df = download_krx_index(at_date, 'KOSPI')
    kosdaq_df = download_krx_index(at_date, 'KOSDAQ')
    return pd.concat([kospi_df, kosdaq_df])

  headers = {             
    "User-Agent": "Mozilla/5.0",
    "Referer": "http://data.krx.co.kr/"
  }
  otp_params = {
      'locale': 'ko_KR',
      'trdDd': at_date,
      'money': '1',
      'csvxls_isNo': 'false',
      'name': 'fileDown',
      'url': 'dbms/MDC/STAT/standard/MDCSTAT03501'
  }
  if market == 'KOSPI':
    otp_params |= { 'mktId': 'STK' }
  elif market == 'KOSDAQ':
    otp_params |= { 'mktId': 'KSQ', 'segTpCd': 'ALL' }

  otp = get_krx_index_otp(otp_params, headers)

  download_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
  download_params = {
      'code': otp
  }
  res = rq.post(download_url, download_params, headers = headers)
  res.encoding = 'euc-kr'
  return pd.read_csv(StringIO(res.text))

In [121]:
import datetime

# 현재 날짜를 YYYYMMDD 형식으로 변환
now = datetime.datetime.now()
# current_date_str = now.strftime('%Y%m%d')
current_date_str='20241002'

In [122]:
df_info = download_krx_sector(current_date_str)

In [123]:

df_index = download_krx_index(current_date_str)

In [124]:
df_info

,표준코드,단축코드,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,액면가,상장주식수
0,KR7095570008,095570,AJ네트웍스보통주,AJ네트웍스,"AJ Networks Co.,Ltd.",2015/08/21,KOSPI,주권,NaN,보통주,1000,45252759
1,KR7006840003,006840,AK홀딩스보통주,AK홀딩스,"AK Holdings, Inc.",1999/08/11,KOSPI,주권,NaN,보통주,5000,13247561
2,KR7282330000,282330,BGF리테일보통주,BGF리테일,BGF Retail,2017/12/08,KOSPI,주권,NaN,보통주,1000,17283906
3,KR7027410000,027410,BGF보통주,BGF,BGF,2014/05/19,KOSPI,주권,NaN,보통주,1000,95716791
4,KR7138930003,138930,BNK금융지주보통주,BNK금융지주,BNK Financial Group Inc.,2011/03/30,KOSPI,주권,NaN,보통주,5000,320436727
...,...,...,...,...,...,...,...,...,...,...,...,...
1750,KR7024060006,024060,흥구석유,흥구석유,HeunguOil,1994/12/07,KOSDAQ,주권,중견기업부,보통주,100,15000000
1751,KR7010240000,010240,흥국,흥국,"HEUNGKUK METALTECH CO.,LTD.",2009/05/12,KOSDAQ,주권,우량기업부,보통주,500,12322696
1752,KR7189980006,189980,흥국에프엔비,흥국에프엔비,"HYUNGKUK F&B Co., Ltd",2015/08/07,KOSDAQ,주권,우량기업부,보통주,100,40137827
1753,KR7037440005,037440,희림종합건축사사무소,희림,Heerim Architects & Planners,2000/02/03,KOSDAQ,주권,우량기업부,보통주,500,13922475


In [125]:
df_index

,종목코드,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,4700,20,0.43,367.0,12.81,781.0,6.02,9326.0,0.50,270,5.74
1,006840,AK홀딩스,13050,160,1.24,2635.0,4.95,NaN,NaN,44339.0,0.29,200,1.53
2,027410,BGF,3540,-20,-0.56,813.0,4.35,650.0,5.44,17286.0,0.20,120,3.39
3,282330,BGF리테일,112900,-1800,-1.57,11337.0,9.96,11788.0,9.58,62197.0,1.82,4100,3.63
4,138930,BNK금융지주,8980,-60,-0.66,1905.0,4.71,2628.0,3.42,31746.0,0.28,510,5.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,024060,흥구석유,18730,1250,7.15,78.0,240.13,NaN,NaN,5440.0,3.44,50,0.27
1731,010240,흥국,4905,10,0.20,1182.0,4.15,NaN,NaN,8598.0,0.57,240,4.89
1732,189980,흥국에프엔비,1782,-50,-2.73,221.0,8.06,222.0,8.01,2449.0,0.73,40,2.24
1733,037440,희림,5070,-130,-2.50,489.0,10.37,NaN,NaN,5583.0,0.91,150,2.96


In [126]:
df_info_mapping = {
    '단축코드': 'SHORT_CODE',
}

df_index_mapping = {
    '종목코드': 'SHORT_CODE',
}

# 컬럼명 매핑 적용
df_info.rename(columns=df_info_mapping, inplace=True)
df_index.rename(columns=df_index_mapping, inplace=True)

In [127]:
merged_df = pd.merge(df_info, df_index, on='SHORT_CODE', how='outer')

In [128]:
merged_df

,표준코드,SHORT_CODE,한글 종목명,한글 종목약명,영문 종목명,상장일,시장구분,증권구분,소속부,주식종류,...,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,KR7000020008,000020,동화약품보통주,동화약품,DongwhaPharm,1976/03/24,KOSPI,주권,NaN,보통주,...,-160.0,-2.06,991.0,7.67,NaN,NaN,13413.0,0.57,180.0,2.37
1,KR7000040006,000040,KR모터스보통주,KR모터스,KR MOTORS,1976/05/25,KOSPI,주권,NaN,보통주,...,-3.0,-0.54,NaN,NaN,NaN,NaN,618.0,0.90,0.0,0.00
2,KR7000050005,000050,경방보통주,경방,Kyungbang,1956/03/03,KOSPI,주권,NaN,보통주,...,-90.0,-1.47,NaN,NaN,NaN,NaN,29623.0,0.20,125.0,2.07
3,KR7000070003,000070,삼양홀딩스보통주,삼양홀딩스,SAMYANGHOLDINGS,1968/12/27,KOSPI,주권,NaN,보통주,...,-1500.0,-1.93,22269.0,3.43,NaN,NaN,257475.0,0.30,3500.0,4.58
4,KR7000071001,000075,삼양홀딩스1우선주,삼양홀딩스우,SAMYANGHOLDINGS(1P),1992/02/21,KOSPI,주권,NaN,구형우선주,...,100.0,0.18,NaN,NaN,NaN,NaN,NaN,NaN,3550.0,6.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,KR8392080006,950170,제이티씨,JTC,JTC Inc.,2018/04/06,KOSDAQ,주식예탁증권,외국기업(소속부없음),보통주,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2710,KR8344390008,950190,고스트스튜디오,고스트스튜디오,GHOST STUDIO,2020/08/18,KOSDAQ,주식예탁증권,외국기업(소속부없음),보통주,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2711,KR8840150005,950200,소마젠,소마젠,"Psomagen, Inc.",2020/07/13,KOSDAQ,주식예탁증권,외국기업(소속부없음),보통주,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2712,KR8702070002,950210,프레스티지바이오파마KDR,프레스티지바이오파마,PRESTIGE BIOPHARMA LIMITED KDR,2021/02/05,KOSPI,주식예탁증권,NaN,보통주,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [129]:
merged_df_column_mapping = {
    '단축코드': 'SHORT_CODE',
    # '서브카테고리' : 'SUB_CATEGORY_ID',
    '한글 종목약명': 'STOCK_NAME',
    '표준코드': 'STANDARD_CODE',
    '시장구분': 'STOCK_EXCHANGE_MARKET',
    '상장주식수': 'MARKET_CAPITALIZATION',
    '시가': 'OPEN_PRICE',
    '종가': 'CLOSED_PRICE',
    '고가': 'HIGH_PRICE',
    '저가': 'LOW_PRICE',
    '거래정지': 'STOCK_TRADE_STATUS',
}


merged_df.rename(columns=merged_df_column_mapping, inplace=True)

In [130]:

real_index = [
 'SHORT_CODE',
 'STOCK_NAME',
 'STANDARD_CODE',
 'STOCK_EXCHANGE_MARKET',
 'MARKET_CAPITALIZATION',
 'EPS',
 'PER',
 'BPS',
 'PBR',
]

merged_df = merged_df[real_index]

In [131]:
for column in [
 'SHORT_CODE',
 'SUB_CATEGORY_ID',
 'STOCK_NAME',
 'STANDARD_CODE',
 'STOCK_EXCHANGE_MARKET',
 'MARKET_CAPITALIZATION',
 'OPEN_PRICE',
 'CLOSED_PRICE',
 'HIGH_PRICE',
 'LOW_PRICE',
 'STOCK_TRADE_STATUS']:
    if column not in merged_df.columns:
        merged_df[column] = 1


C:\Users\David\AppData\Local\Temp\ipykernel_36904\1445798947.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[column] = 1
C:\Users\David\AppData\Local\Temp\ipykernel_36904\1445798947.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[column] = 1
C:\Users\David\AppData\Local\Temp\ipykernel_36904\1445798947.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [132]:

columns_to_keep = [
 'SHORT_CODE',
 'SUB_CATEGORY_ID',
 'STOCK_NAME',
 'STANDARD_CODE',
 'STOCK_EXCHANGE_MARKET',
 'EPS',
 'PER',
 'BPS',
 'PBR',
 'OPEN_PRICE',
 'CLOSED_PRICE',
 'HIGH_PRICE',
 'LOW_PRICE',
 'MARKET_CAPITALIZATION',
 'STOCK_TRADE_STATUS'
]

merged_df = merged_df[columns_to_keep]

In [133]:
merged_df

,SHORT_CODE,SUB_CATEGORY_ID,STOCK_NAME,STANDARD_CODE,STOCK_EXCHANGE_MARKET,EPS,PER,BPS,PBR,OPEN_PRICE,CLOSED_PRICE,HIGH_PRICE,LOW_PRICE,MARKET_CAPITALIZATION,STOCK_TRADE_STATUS
0,000020,1,동화약품,KR7000020008,KOSPI,991.0,7.67,13413.0,0.57,1,1,1,1,27931470,1
1,000040,1,KR모터스,KR7000040006,KOSPI,NaN,NaN,618.0,0.90,1,1,1,1,60132868,1
2,000050,1,경방,KR7000050005,KOSPI,NaN,NaN,29623.0,0.20,1,1,1,1,27415270,1
3,000070,1,삼양홀딩스,KR7000070003,KOSPI,22269.0,3.43,257475.0,0.30,1,1,1,1,8564271,1
4,000075,1,삼양홀딩스우,KR7000071001,KOSPI,NaN,NaN,NaN,NaN,1,1,1,1,304058,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,950170,1,JTC,KR8392080006,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,51746348,1
2710,950190,1,고스트스튜디오,KR8344390008,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,13579892,1
2711,950200,1,소마젠,KR8840150005,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,19236053,1
2712,950210,1,프레스티지바이오파마,KR8702070002,KOSPI,NaN,NaN,NaN,NaN,1,1,1,1,60096155,1


In [134]:
for column in [
 'SHORT_CODE',
 'SUB_CATEGORY_ID',
 'STOCK_NAME',
 'STANDARD_CODE',
 'STOCK_EXCHANGE_MARKET',
 'MARKET_CAPITALIZATION',
 'OPEN_PRICE',
 'CLOSED_PRICE',
 'HIGH_PRICE',
 'LOW_PRICE',
 'STOCK_TRADE_STATUS']:
    if column not in merged_df.columns:
        merged_df[column] = -1


In [135]:
merged_df

,SHORT_CODE,SUB_CATEGORY_ID,STOCK_NAME,STANDARD_CODE,STOCK_EXCHANGE_MARKET,EPS,PER,BPS,PBR,OPEN_PRICE,CLOSED_PRICE,HIGH_PRICE,LOW_PRICE,MARKET_CAPITALIZATION,STOCK_TRADE_STATUS
0,000020,1,동화약품,KR7000020008,KOSPI,991.0,7.67,13413.0,0.57,1,1,1,1,27931470,1
1,000040,1,KR모터스,KR7000040006,KOSPI,NaN,NaN,618.0,0.90,1,1,1,1,60132868,1
2,000050,1,경방,KR7000050005,KOSPI,NaN,NaN,29623.0,0.20,1,1,1,1,27415270,1
3,000070,1,삼양홀딩스,KR7000070003,KOSPI,22269.0,3.43,257475.0,0.30,1,1,1,1,8564271,1
4,000075,1,삼양홀딩스우,KR7000071001,KOSPI,NaN,NaN,NaN,NaN,1,1,1,1,304058,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,950170,1,JTC,KR8392080006,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,51746348,1
2710,950190,1,고스트스튜디오,KR8344390008,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,13579892,1
2711,950200,1,소마젠,KR8840150005,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,19236053,1
2712,950210,1,프레스티지바이오파마,KR8702070002,KOSPI,NaN,NaN,NaN,NaN,1,1,1,1,60096155,1


In [136]:
df_i

,SHORT_CODE,종목명,종가,대비,등락률,EPS,PER,선행 EPS,선행 PER,BPS,PBR,주당배당금,배당수익률
0,095570,AJ네트웍스,4700,20,0.43,367.0,12.81,781.0,6.02,9326.0,0.50,270,5.74
1,006840,AK홀딩스,13050,160,1.24,2635.0,4.95,NaN,NaN,44339.0,0.29,200,1.53
2,027410,BGF,3540,-20,-0.56,813.0,4.35,650.0,5.44,17286.0,0.20,120,3.39
3,282330,BGF리테일,112900,-1800,-1.57,11337.0,9.96,11788.0,9.58,62197.0,1.82,4100,3.63
4,138930,BNK금융지주,8980,-60,-0.66,1905.0,4.71,2628.0,3.42,31746.0,0.28,510,5.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1730,024060,흥구석유,18730,1250,7.15,78.0,240.13,NaN,NaN,5440.0,3.44,50,0.27
1731,010240,흥국,4905,10,0.20,1182.0,4.15,NaN,NaN,8598.0,0.57,240,4.89
1732,189980,흥국에프엔비,1782,-50,-2.73,221.0,8.06,222.0,8.01,2449.0,0.73,40,2.24
1733,037440,희림,5070,-130,-2.50,489.0,10.37,NaN,NaN,5583.0,0.91,150,2.96


In [137]:
merged_df 

,SHORT_CODE,SUB_CATEGORY_ID,STOCK_NAME,STANDARD_CODE,STOCK_EXCHANGE_MARKET,EPS,PER,BPS,PBR,OPEN_PRICE,CLOSED_PRICE,HIGH_PRICE,LOW_PRICE,MARKET_CAPITALIZATION,STOCK_TRADE_STATUS
0,000020,1,동화약품,KR7000020008,KOSPI,991.0,7.67,13413.0,0.57,1,1,1,1,27931470,1
1,000040,1,KR모터스,KR7000040006,KOSPI,NaN,NaN,618.0,0.90,1,1,1,1,60132868,1
2,000050,1,경방,KR7000050005,KOSPI,NaN,NaN,29623.0,0.20,1,1,1,1,27415270,1
3,000070,1,삼양홀딩스,KR7000070003,KOSPI,22269.0,3.43,257475.0,0.30,1,1,1,1,8564271,1
4,000075,1,삼양홀딩스우,KR7000071001,KOSPI,NaN,NaN,NaN,NaN,1,1,1,1,304058,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,950170,1,JTC,KR8392080006,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,51746348,1
2710,950190,1,고스트스튜디오,KR8344390008,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,13579892,1
2711,950200,1,소마젠,KR8840150005,KOSDAQ,NaN,NaN,NaN,NaN,1,1,1,1,19236053,1
2712,950210,1,프레스티지바이오파마,KR8702070002,KOSPI,NaN,NaN,NaN,NaN,1,1,1,1,60096155,1


In [138]:
merged_df['STOCK_EXCHANGE_MARKET'] = merged_df['STOCK_EXCHANGE_MARKET'].map({'KOSPI': "0", 'KOSDAQ': "1"})

In [139]:
merged_df

,SHORT_CODE,SUB_CATEGORY_ID,STOCK_NAME,STANDARD_CODE,STOCK_EXCHANGE_MARKET,EPS,PER,BPS,PBR,OPEN_PRICE,CLOSED_PRICE,HIGH_PRICE,LOW_PRICE,MARKET_CAPITALIZATION,STOCK_TRADE_STATUS
0,000020,1,동화약품,KR7000020008,0,991.0,7.67,13413.0,0.57,1,1,1,1,27931470,1
1,000040,1,KR모터스,KR7000040006,0,NaN,NaN,618.0,0.90,1,1,1,1,60132868,1
2,000050,1,경방,KR7000050005,0,NaN,NaN,29623.0,0.20,1,1,1,1,27415270,1
3,000070,1,삼양홀딩스,KR7000070003,0,22269.0,3.43,257475.0,0.30,1,1,1,1,8564271,1
4,000075,1,삼양홀딩스우,KR7000071001,0,NaN,NaN,NaN,NaN,1,1,1,1,304058,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2709,950170,1,JTC,KR8392080006,1,NaN,NaN,NaN,NaN,1,1,1,1,51746348,1
2710,950190,1,고스트스튜디오,KR8344390008,1,NaN,NaN,NaN,NaN,1,1,1,1,13579892,1
2711,950200,1,소마젠,KR8840150005,1,NaN,NaN,NaN,NaN,1,1,1,1,19236053,1
2712,950210,1,프레스티지바이오파마,KR8702070002,0,NaN,NaN,NaN,NaN,1,1,1,1,60096155,1


In [140]:
from sqlalchemy import create_engine
from sqlalchemy import text
# MySQL 데이터베이스 연결 설정
db_username = DB_USERNAME
db_password = DB_PASSWORD
db_host = DB_HOST
db_name = DB_SCHEME

# SQLAlchemy 엔진 생성
engine = create_engine(f'mysql+pymysql://{db_username}:{db_password}@{db_host}/{db_name}')

merged_df.to_sql(name='TEMP_STOCK', con=engine, if_exists='replace', index=False)

# 업데이트 쿼리 작성
update_query = """
INSERT INTO STOCK (SHORT_CODE, SUB_CATEGORY_ID, STOCK_NAME, STANDARD_CODE, STOCK_EXCHANGE_MARKET, 
                   EPS, PER, BPS, PBR, MARKET_CAPITALIZATION, OPEN_PRICE, CLOSED_PRICE, 
                   HIGH_PRICE, LOW_PRICE, STOCK_TRADE_STATUS, UPDATED_AT)
VALUES 
    (#{shortCode}, #{subCategoryId}, #{stockName}, #{standardCode}, #{stockExchangeMarket},
     #{eps}, #{per}, #{bps}, #{pbr}, #{marketCapitalization}, #{openPrice}, #{closedPrice},
     #{highPrice}, #{lowPrice}, #{stockTradeStatus}, NOW())
ON DUPLICATE KEY UPDATE 
    SUB_CATEGORY_ID = VALUES(SUB_CATEGORY_ID),
    STOCK_NAME = VALUES(STOCK_NAME),
    STANDARD_CODE = VALUES(STANDARD_CODE),
    STOCK_EXCHANGE_MARKET = VALUES(STOCK_EXCHANGE_MARKET),
    EPS = VALUES(EPS),
    PER = VALUES(PER),
    BPS = VALUES(BPS),
    PBR = VALUES(PBR),
    MARKET_CAPITALIZATION = VALUES(MARKET_CAPITALIZATION),
    OPEN_PRICE = VALUES(OPEN_PRICE),
    CLOSED_PRICE = VALUES(CLOSED_PRICE),
    HIGH_PRICE = VALUES(HIGH_PRICE),
    LOW_PRICE = VALUES(LOW_PRICE),
    STOCK_TRADE_STATUS = VALUES(STOCK_TRADE_STATUS),
    UPDATED_AT = NOW();
"""

# SQL 쿼리 실행
with engine.connect() as connection:
    connection.execute(text(update_query))

# 임시 테이블 삭제
with engine.connect() as connection:
    connection.execute(text("DROP TABLE temp_stock"))


ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ON CONFLICT (SHORT_CODE) \n    SUB_CATEGORY_ID = VALUES(SUB_CATEGORY_ID),\n    STO' at line 8")
[SQL: 
INSERT INTO STOCK (SHORT_CODE, SUB_CATEGORY_ID, STOCK_NAME, STANDARD_CODE, STOCK_EXCHANGE_MARKET, 
                   EPS, PER, BPS, PBR, MARKET_CAPITALIZATION, OPEN_PRICE, CLOSED_PRICE, 
                   HIGH_PRICE, LOW_PRICE, STOCK_TRADE_STATUS, UPDATED_AT)
VALUES 
    (#{shortCode}, #{subCategoryId}, #{stockName}, #{standardCode}, #{stockExchangeMarket},
     #{eps}, #{per}, #{bps}, #{pbr}, #{marketCapitalization}, #{openPrice}, #{closedPrice},
     #{highPrice}, #{lowPrice}, #{stockTradeStatus}, NOW())
ON CONFLICT (SHORT_CODE) 
    SUB_CATEGORY_ID = VALUES(SUB_CATEGORY_ID),
    STOCK_NAME = VALUES(STOCK_NAME),
    STANDARD_CODE = VALUES(STANDARD_CODE),
    STOCK_EXCHANGE_MARKET = VALUES(STO